In [1]:
import torch as t
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
import torch.nn.functional as F


from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pandas as pd

In [2]:
train_data = 'https://github.com/adam-213/MVO2022/raw/main/Data/HDP.txt'
test_data = 'https://github.com/adam-213/MVO2022/raw/main/Data/HDP_test.txt'
dev = 'cuda:0' if t.cuda.is_available() else 'cpu'
dev = 'cpu'

In [3]:
class dataset(Dataset):

  def __init__(self,file_name):
    super().__init__()
    data=pd.read_csv(file_name)
    x=data.iloc[:,[0,2,3]].values
    y=data.iloc[:,1].values

    y = y.reshape(-1, 1)
    
    
    self.x_train=t.tensor(x,dtype=t.float32,device=dev)
    self.y_train=t.tensor(y.reshape(-1),dtype=t.long,device=dev)
    print(self.y_train)
    
  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

data = dataset(train_data)
validation_data = dataset(test_data)

tensor([ 267,  277,  294,  306,  320,  328,  335,  339,  341,  342,  347,  360,
         366,  369,  368,  363,  362,  362,  366,  375,  388,  396,  404,  411,
         412,  418,  423,  432,  440,  442,  449,  444,  436,  440,  453,  467,
         477,  490,  489,  495,  506,  506,  508,  504,  508,  519,  528,  542,
         554,  562,  568,  574,  582,  590,  601,  612,  625,  634,  642,  648,
         668,  681,  696,  717,  738,  750,  760,  774,  780,  788,  805,  823,
         841,  867,  884,  900,  921,  937,  955,  962,  972,  986, 1003, 1009,
        1049, 1068, 1086, 1105, 1142, 1171, 1196, 1233, 1283, 1307, 1337, 1376,
        1387, 1423, 1451, 1473, 1479, 1516, 1578, 1621, 1672, 1698, 1729, 1772,
        1834, 1895, 1954, 1988, 2031, 2139, 2202, 2281, 2335, 2377, 2454, 2502,
        2572, 2578, 2639, 2736])
tensor([2875, 2918, 3009, 3027, 3026, 3061, 3080, 3109, 3173, 3267, 3346, 3431])


In [4]:
train_size = int(0.8 * len(data))
test_size = len(data) - train_size

In [5]:
train_data, test_data = random_split(data, [train_size, test_size])
train_loader=DataLoader(train_data,batch_size=64,shuffle=False)
test_loader=DataLoader(test_data,batch_size=64,shuffle=False)
validation_loader = DataLoader(validation_data,batch_size=4,shuffle=False)

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3, 768)
        self.fc2 = nn.Linear(768, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)
        self.d = nn.Dropout(0.2)
        
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.d(self.fc1(x)))
        x = F.relu(self.d(self.fc2(x)))
        x = F.relu(self.d(self.fc3(x)))
        x = F.relu(self.d(self.fc4(x)))
        x = F.log_softmax(self.fc5(x), dim=1)
        return x

In [7]:
model = Classifier().to(dev)
criterion = nn.NLLLoss().to(dev)
optimizer = t.optim.Adam(model.parameters(), lr=0.003)
epochs = 500

In [8]:
CUDA_LAUNCH_BLOCKING=1.
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        images,labels = images.to(dev),labels.to(dev)
        if dev != "cpu":
            with t.cuda.amp.autocast():
                log_ps = model(images)
                loss = criterion(log_ps, labels).to(dev)
        else:
            log_ps = model(images)
            loss = criterion(log_ps, labels).to(dev)
            
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    else:
        if e % 5 == 0:
            with torch.no_grad():
                images, labels = next(iter(test_loader))
                ps = t.exp(model(images.to(dev)))
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.to(dev).view(*top_class.shape)
                accuracy = t.mean(equals.type(torch.FloatTensor))
                print(f'Accuracy: {accuracy.item()*100}%',end='\r')

IndexError: Target 668 is out of bounds.